# Problem statement

Using providers personal information, payment history and drug prescription detect the fraudulent in medicare system with the help of 
previously excluded provider list available on CMS website.


In this dataset there is an information about number of different drugs prescribed.The sum of the number of prescriptions.The sum of the number of days prescribed.The sum of the total cost.
The variance of these three sums quantities.
The maximum of these three sums quantities
The Prescribing pattern features to figure out the prescribe-related fraud

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# reading the data
data=pd.read_csv("./datasets/PartD_Prescriber_PUF_NPI_Drug_17.txt", sep='\t')

In [3]:
data.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,...,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65
0,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE,NaN,...,13.0,390,59.21,NaN,*,NaN,*,NaN,NaN,NaN
1,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,ATORVASTATIN CALCIUM,ATORVASTATIN CALCIUM,11.0,...,27.0,765,259.48,NaN,#,NaN,#,NaN,NaN,NaN
2,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,CEPHALEXIN,CEPHALEXIN,17.0,...,17.0,123,98.99,NaN,#,NaN,#,NaN,NaN,NaN
3,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,CIPROFLOXACIN HCL,CIPROFLOXACIN HCL,11.0,...,12.0,95,120.43,NaN,*,NaN,*,NaN,NaN,NaN
4,1003000126,ENKESHAFI,ARDALAN,CUMBERLAND,MD,Internal Medicine,S,DOXYCYCLINE HYCLATE,DOXYCYCLINE HYCLATE,17.0,...,17.0,105,300.76,NaN,#,NaN,#,NaN,NaN,NaN


In [4]:
data.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_city', 'nppes_provider_state', 'specialty_description',
       'description_flag', 'drug_name', 'generic_name', 'bene_count',
       'total_claim_count', 'total_30_day_fill_count', 'total_day_supply',
       'total_drug_cost', 'bene_count_ge65', 'bene_count_ge65_suppress_flag',
       'total_claim_count_ge65', 'ge65_suppress_flag',
       'total_30_day_fill_count_ge65', 'total_day_supply_ge65',
       'total_drug_cost_ge65'],
      dtype='object')

In [5]:
#selecting some important columns

data=data[['npi','nppes_provider_city','nppes_provider_state', 
                                               'nppes_provider_last_org_name', 
                                               'nppes_provider_first_name', 
                                               'specialty_description',
                                                'drug_name', 'generic_name',
                                                'total_drug_cost',
                                               'total_claim_count',
                                               'total_day_supply']]

In [6]:
#renaming the column names

data=data.rename(columns={'npi':'NPI','nppes_provider_first_name':'first_name', 'nppes_provider_last_org_name':'last_name',
             'nppes_provider_city':'city','nppes_provider_state':'state'})

In [7]:
data.head()

,NPI,city,state,last_name,first_name,specialty_description,drug_name,generic_name,total_drug_cost,total_claim_count,total_day_supply
0,1003000126,CUMBERLAND,MD,ENKESHAFI,ARDALAN,Internal Medicine,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE,59.21,13,390
1,1003000126,CUMBERLAND,MD,ENKESHAFI,ARDALAN,Internal Medicine,ATORVASTATIN CALCIUM,ATORVASTATIN CALCIUM,259.48,27,765
2,1003000126,CUMBERLAND,MD,ENKESHAFI,ARDALAN,Internal Medicine,CEPHALEXIN,CEPHALEXIN,98.99,17,123
3,1003000126,CUMBERLAND,MD,ENKESHAFI,ARDALAN,Internal Medicine,CIPROFLOXACIN HCL,CIPROFLOXACIN HCL,120.43,12,95
4,1003000126,CUMBERLAND,MD,ENKESHAFI,ARDALAN,Internal Medicine,DOXYCYCLINE HYCLATE,DOXYCYCLINE HYCLATE,300.76,17,105


In [8]:
#making firstname and last name as name

data["Name"]=data["first_name"]+" "+data["last_name"]

In [9]:
# all npi over here is unique
data["NPI"].nunique()

918009

In [10]:
data.shape

(25209130, 12)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209130 entries, 0 to 25209129
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   NPI                    int64  
 1   city                   object 
 2   state                  object 
 3   last_name              object 
 4   first_name             object 
 5   specialty_description  object 
 6   drug_name              object 
 7   generic_name           object 
 8   total_drug_cost        float64
 9   total_claim_count      int64  
 10  total_day_supply       int64  
 11  Name                   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 2.3+ GB


In [12]:
data.isnull().sum()


NPI                        0
city                       0
state                      0
last_name                471
first_name               351
specialty_description      0
drug_name                  0
generic_name               0
total_drug_cost            0
total_claim_count          0
total_day_supply           0
Name                     822
dtype: int64

In [13]:
#imputing null values 

data=data.fillna("Unknown")

In [14]:
data["last_name"].nunique()

224649

In [15]:
data=data.drop(columns=["first_name","last_name"])

In [16]:
data.columns

Index(['NPI', 'city', 'state', 'specialty_description', 'drug_name',
       'generic_name', 'total_drug_cost', 'total_claim_count',
       'total_day_supply', 'Name'],
      dtype='object')

In [17]:
data=data[['NPI', 'Name','city', 'state', 'specialty_description', 'drug_name',
       'generic_name', 'total_drug_cost', 'total_claim_count',
       'total_day_supply']]

In [18]:
data.isnull().sum()

NPI                      0
Name                     0
city                     0
state                    0
specialty_description    0
drug_name                0
generic_name             0
total_drug_cost          0
total_claim_count        0
total_day_supply         0
dtype: int64

In [19]:
#seperating categorical columns

cat_data=data[["NPI","Name","city","state","specialty_description"]]

In [20]:
#all numerical columns

num_data=data[["NPI","total_drug_cost","total_claim_count","total_day_supply"]]

In [21]:
cat_data

,NPI,Name,city,state,specialty_description
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine
1,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine
2,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine
3,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine
4,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine
...,...,...,...,...,...
25209125,1992999882,MARISSA RUDZINSKI-WALTER,WILKES BARRE,PA,Physician Assistant
25209126,1992999882,MARISSA RUDZINSKI-WALTER,WILKES BARRE,PA,Physician Assistant
25209127,1992999882,MARISSA RUDZINSKI-WALTER,WILKES BARRE,PA,Physician Assistant
25209128,1992999882,MARISSA RUDZINSKI-WALTER,WILKES BARRE,PA,Physician Assistant


In [22]:
#we can see there are duplicates 

cat_data=cat_data.drop_duplicates()


In [23]:
cat_data

,NPI,Name,city,state,specialty_description
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine
16,1003000142,RASHID KHALIL,TOLEDO,OH,Anesthesiology
38,1003000167,JULIO ESCOBAR,DAYTON,NV,Dentist
40,1003000282,ROSIE BLAKEMORE,NASHVILLE,TN,Nurse Practitioner
42,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice
...,...,...,...,...,...
25209081,1992999759,DINO SORIANO,GUYTON,GA,Nurse Practitioner
25209110,1992999791,SHELIA THOMAS,MEMPHIS,TN,Registered Nurse
25209113,1992999825,GEOFFREY DESCHENES,SEATTLE,WA,Otolaryngology
25209121,1992999833,L. NOAH SHAW,NEW YORK,NY,Psychoanalyst


In [24]:
num_data

,NPI,total_drug_cost,total_claim_count,total_day_supply
0,1003000126,59.21,13,390
1,1003000126,259.48,27,765
2,1003000126,98.99,17,123
3,1003000126,120.43,12,95
4,1003000126,300.76,17,105
...,...,...,...,...
25209125,1992999882,103.74,13,335
25209126,1992999882,120.76,18,536
25209127,1992999882,91.96,12,356
25209128,1992999882,102.30,18,515


In [25]:
#Grouping numerical columns by NPI

num_data=num_data.groupby("NPI").sum()

In [26]:
num_data=num_data.reset_index()

In [27]:
data=pd.merge(cat_data,num_data,on="NPI")

In [28]:
data.head()

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053
1,1003000142,RASHID KHALIL,TOLEDO,OH,Anesthesiology,113868.77,1846,55569
2,1003000167,JULIO ESCOBAR,DAYTON,NV,Dentist,151.75,38,157
3,1003000282,ROSIE BLAKEMORE,NASHVILLE,TN,Nurse Practitioner,659.99,54,1472
4,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901


To merging the above dataset, I am reading opioid dataset that I have cleaned earlier

In [29]:
data_1=pd.read_csv("./datasets/opiod.csv",index_col=0)

/Users/umangimistry/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Merging the opioid datasets with the prescriber datasets

In [30]:
# merging the datasets using common column NPI

df=pd.merge(data,data_1,on="NPI")

In [31]:
df.head()

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,Long-Acting Opioid Claim Count,Long-Acting Opioid Prescribing Rate
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053,25.0,3.69,0.0,0.00
1,1003000142,RASHID KHALIL,TOLEDO,OH,Anesthesiology,113868.77,1846,55569,1040.0,53.44,200.0,19.23
2,1003000167,JULIO ESCOBAR,DAYTON,NV,Dentist,151.75,38,157,14.0,25.45,0.0,0.00
3,1003000282,ROSIE BLAKEMORE,NASHVILLE,TN,Nurse Practitioner,659.99,54,1472,0.0,0.00,0.0,0.00
4,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,15.0,46.88


In [32]:
df=df.drop_duplicates()

Reading physician yearly payment dataset which I have already cleaned in other notebook

In [33]:
payment=pd.read_csv("./datasets/payment.csv",index_col=0)

In [34]:
payment

,Name,Total_Payment_Sum
0,01SOHEIR AZER,12.64
1,A AKBAR,366.07
2,A ALBALA,112622.87
3,A ALICEA,362.77
4,A AMINIAN,4690.72
...,...,...
546473,ZYGMUNT ORZECHOWSKI,19.34
546474,ZYLKIA RODRIGUEZ,238.55
546475,ZYNIA PUA-VINES,104.33
546476,ZYNNIA ZAFRA,923.82


I am going to merge the payment data with prescriber and opioid prescriber dataset

In [35]:
df1 = pd.merge (df,payment, on = 'Name')

In [36]:
df1.head()

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,Long-Acting Opioid Claim Count,Long-Acting Opioid Prescribing Rate,Total_Payment_Sum
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053,25.0,3.69,0.0,0.00,25.51
1,1003000142,RASHID KHALIL,TOLEDO,OH,Anesthesiology,113868.77,1846,55569,1040.0,53.44,200.0,19.23,1979.06
2,1003000167,JULIO ESCOBAR,DAYTON,NV,Dentist,151.75,38,157,14.0,25.45,0.0,0.00,127.16
3,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,15.0,46.88,1052.15
4,1558452516,DAVID GIRARDI,SYRACUSE,NY,Optometry,18165.20,125,7499,0.0,0.00,0.0,0.00,1052.15


So now I have the list of providers with their unique NPI numbers and all drug prescriber, total payment and opioid prescribe data

In [37]:
df1.columns

Index(['NPI', 'Name', 'city', 'state', 'specialty_description',
       'total_drug_cost', 'total_claim_count', 'total_day_supply',
       'Opioid Claim Count', 'Opioid Prescribing Rate',
       'Long-Acting Opioid Claim Count', 'Long-Acting Opioid Prescribing Rate',
       'Total_Payment_Sum'],
      dtype='object')

In [38]:
df1["NPI"].nunique()

503248

Lastly, I am adding the supplier and provider's aggregated payment and claim data

In [42]:
df4=pd.read_csv("./datasets/supplier1_data.csv",index_col=0)

In [43]:
df=pd.merge(df1,df4,on="NPI")

In [44]:
df.head()

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,...,Long-Acting Opioid Prescribing Rate,Total_Payment_Sum,Supplier Rental Indicator,Number of Suppliers,Number of Supplier Beneficiaries,Number of Supplier Claims,Average Supplier Submitted Charge,Average Supplier Medicare Allow Amount,Average Supplier Medicare Payment Amount,Average Supplier Medicare Standard Amount
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053,25.0,3.69,...,0.00,25.51,2,11,32.0,214,405.929470,95.977478,70.156976,64.189152
1,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,...,46.88,1052.15,1,2,0.0,12,28.000000,4.972500,3.900000,3.597500
2,1003000522,FREDERICK WEIGAND,DELTONA,FL,Family Practice,148671.74,4369,301669,128.0,2.57,...,0.00,436.34,1,29,43.0,109,245.186143,92.320000,67.667474,62.203188
3,1003000597,DAE KIM,TULSA,OK,Urology,62041.35,689,28338,92.0,11.93,...,0.00,2406.56,0,3,0.0,14,16.469722,7.380000,5.463611,5.463611
4,1720221062,DAE KIM,BRYN MAWR,PA,Hospitalist,174.42,11,71,0.0,0.00,...,0.00,2406.56,1,1,0.0,11,85.000000,37.000000,23.735455,24.259091


In [45]:
df.isnull().sum()

NPI                                          0
Name                                         0
city                                         0
state                                        0
specialty_description                        0
total_drug_cost                              0
total_claim_count                            0
total_day_supply                             0
Opioid Claim Count                           0
Opioid Prescribing Rate                      0
Long-Acting Opioid Claim Count               0
Long-Acting Opioid Prescribing Rate          0
Total_Payment_Sum                            0
Supplier Rental Indicator                    0
Number of Suppliers                          0
Number of Supplier Beneficiaries             0
Number of Supplier Claims                    0
Average Supplier Submitted Charge            0
Average Supplier Medicare Allow Amount       0
Average Supplier Medicare Payment Amount     0
Average Supplier Medicare Standard Amount    0
dtype: int64

Now, I have a dataset of updated excluded providers so I will use them to find fraud providers and physician in my dataset

In [46]:
data_1=pd.read_csv("./datasets/UPDATED.csv")

/Users/umangimistry/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [47]:
data_1.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,NaN,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,20200319,0,0,NaN
1,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
2,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
3,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN
4,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN


This datasets has total 73704 providers who are fraud but not all have provided their NPI numbers. So I will only use those who has their NPI numbers.

In [48]:
npi=data_1[["NPI","EXCLTYPE"]]
npi

,NPI,EXCLTYPE
0,0,1128a1
1,0,1128a1
2,0,1128b7
3,1922348218,1128a1
4,0,1128b5
...,...,...
73699,0,1128b4
73700,0,1128b4
73701,0,1128b4
73702,0,1128a2


In [49]:
npi.rename(columns = {"EXCLTYPE":'is_fraud'}, inplace = True)

/Users/umangimistry/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


I am putting 1 if there is NPI numbers otherwise 0

In [50]:
npi.loc[npi["NPI"] != 0, "is_fraud"] = 1
npi.loc[npi["NPI"] == 0, "is_fraud"] = 0

/Users/umangimistry/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


So there are total 5588 frauds with their NPI numbers

In [51]:
npi["is_fraud"].value_counts()

0    68116
1     5588
Name: is_fraud, dtype: int64

In [52]:
fraud=npi[npi["is_fraud"]==1]

In [53]:
fraud

,NPI,is_fraud
3,1922348218,1
23,1942476080,1
27,1275600959,1
30,1891731758,1
58,1851631543,1
...,...,...
73661,1124024435,1
73668,1194930149,1
73678,1578637385,1
73683,1558366815,1


Let's check the frauds in our dataset 

In [54]:
df.head()

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,...,Long-Acting Opioid Prescribing Rate,Total_Payment_Sum,Supplier Rental Indicator,Number of Suppliers,Number of Supplier Beneficiaries,Number of Supplier Claims,Average Supplier Submitted Charge,Average Supplier Medicare Allow Amount,Average Supplier Medicare Payment Amount,Average Supplier Medicare Standard Amount
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053,25.0,3.69,...,0.00,25.51,2,11,32.0,214,405.929470,95.977478,70.156976,64.189152
1,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,...,46.88,1052.15,1,2,0.0,12,28.000000,4.972500,3.900000,3.597500
2,1003000522,FREDERICK WEIGAND,DELTONA,FL,Family Practice,148671.74,4369,301669,128.0,2.57,...,0.00,436.34,1,29,43.0,109,245.186143,92.320000,67.667474,62.203188
3,1003000597,DAE KIM,TULSA,OK,Urology,62041.35,689,28338,92.0,11.93,...,0.00,2406.56,0,3,0.0,14,16.469722,7.380000,5.463611,5.463611
4,1720221062,DAE KIM,BRYN MAWR,PA,Hospitalist,174.42,11,71,0.0,0.00,...,0.00,2406.56,1,1,0.0,11,85.000000,37.000000,23.735455,24.259091


Merging our dataset with fraud provider's dataset

In [55]:
df= pd.merge(df,fraud, how ='left',on = "NPI")

In [56]:
df

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,...,Total_Payment_Sum,Supplier Rental Indicator,Number of Suppliers,Number of Supplier Beneficiaries,Number of Supplier Claims,Average Supplier Submitted Charge,Average Supplier Medicare Allow Amount,Average Supplier Medicare Payment Amount,Average Supplier Medicare Standard Amount,is_fraud
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053,25.0,3.69,...,25.51,2,11,32.0,214,405.929470,95.977478,70.156976,64.189152,NaN
1,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,...,1052.15,1,2,0.0,12,28.000000,4.972500,3.900000,3.597500,1
2,1003000522,FREDERICK WEIGAND,DELTONA,FL,Family Practice,148671.74,4369,301669,128.0,2.57,...,436.34,1,29,43.0,109,245.186143,92.320000,67.667474,62.203188,NaN
3,1003000597,DAE KIM,TULSA,OK,Urology,62041.35,689,28338,92.0,11.93,...,2406.56,0,3,0.0,14,16.469722,7.380000,5.463611,5.463611,NaN
4,1720221062,DAE KIM,BRYN MAWR,PA,Hospitalist,174.42,11,71,0.0,0.00,...,2406.56,1,1,0.0,11,85.000000,37.000000,23.735455,24.259091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178084,1992996458,PHILIP MATAVERDE,OSAGE BEACH,MO,Sleep Medicine,17601.87,380,12677,0.0,0.00,...,32.57,6,262,2257.0,5507,4546.366891,977.103312,720.427468,652.541468,NaN
178085,1992996631,FUNLOLA FAMUYIWA,REISTERSTOWN,MD,Pulmonary Disease,485479.89,1452,46382,0.0,0.00,...,1067.61,8,218,1100.0,3440,4707.572283,997.803458,755.375066,701.002942,NaN
178086,1992997647,MATTHEW ROBERGE,WHITEHALL,MI,Family Practice,235188.91,4073,195320,595.0,12.43,...,1669.47,3,25,13.0,134,815.777068,186.771935,136.680719,133.279903,NaN
178087,1992998645,SESHASREE MARUPUDI,SHENANDOAH,TX,Internal Medicine,301500.40,5215,337688,188.0,3.20,...,289.21,4,40,11.0,195,788.486078,208.440869,147.029627,146.017574,NaN


In [57]:
# filling 0 in is_fraud column where there is no fraud

df["is_fraud"]=df["is_fraud"].fillna(0)

In [58]:
df.head()

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,...,Total_Payment_Sum,Supplier Rental Indicator,Number of Suppliers,Number of Supplier Beneficiaries,Number of Supplier Claims,Average Supplier Submitted Charge,Average Supplier Medicare Allow Amount,Average Supplier Medicare Payment Amount,Average Supplier Medicare Standard Amount,is_fraud
0,1003000126,ARDALAN ENKESHAFI,CUMBERLAND,MD,Internal Medicine,12374.60,385,8053,25.0,3.69,...,25.51,2,11,32.0,214,405.929470,95.977478,70.156976,64.189152,0
1,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,...,1052.15,1,2,0.0,12,28.000000,4.972500,3.900000,3.597500,1
2,1003000522,FREDERICK WEIGAND,DELTONA,FL,Family Practice,148671.74,4369,301669,128.0,2.57,...,436.34,1,29,43.0,109,245.186143,92.320000,67.667474,62.203188,0
3,1003000597,DAE KIM,TULSA,OK,Urology,62041.35,689,28338,92.0,11.93,...,2406.56,0,3,0.0,14,16.469722,7.380000,5.463611,5.463611,0
4,1720221062,DAE KIM,BRYN MAWR,PA,Hospitalist,174.42,11,71,0.0,0.00,...,2406.56,1,1,0.0,11,85.000000,37.000000,23.735455,24.259091,0


To use this final dataset for modeling, I am saving it as a csv file

In [59]:
df.to_csv("./datasets/final_data.csv")

Let's check the frauds in our dataset

In [60]:
df["is_fraud"].value_counts()

0    177973
1       116
Name: is_fraud, dtype: int64

In [61]:
df.merge(fraud)

,NPI,Name,city,state,specialty_description,total_drug_cost,total_claim_count,total_day_supply,Opioid Claim Count,Opioid Prescribing Rate,...,Total_Payment_Sum,Supplier Rental Indicator,Number of Suppliers,Number of Supplier Beneficiaries,Number of Supplier Claims,Average Supplier Submitted Charge,Average Supplier Medicare Allow Amount,Average Supplier Medicare Payment Amount,Average Supplier Medicare Standard Amount,is_fraud
0,1003000407,DAVID GIRARDI,BROOKVILLE,PA,Family Practice,73501.23,2286,41901,32.0,1.15,...,1052.15,1,2,0.0,12,28.000000,4.972500,3.900000,3.597500,1
1,1003127655,MOUSTAFA ABOSHADY,LOS ALAMITOS,CA,Internal Medicine,168732.00,4663,90764,248.0,4.55,...,63.66,9,66,40.0,521,753.857921,388.559379,303.362041,299.128562,1
2,1104898634,MICHAEL SMITH,MOUNT HOLLY,NC,Family Practice,511093.81,10885,368903,816.0,7.11,...,75341.70,4,57,42.0,344,839.781184,186.938129,133.202212,128.381894,1
3,1003850603,STEVEN OWENS,MASON,MI,Family Practice,243563.77,4272,167529,852.0,17.96,...,313.76,0,6,0.0,14,47.963824,8.320000,4.796471,4.796471,1
4,1396772737,HOWARD DIAMOND,RICHARDSON,TX,Physical Medicine and Rehabilitation,2138300.50,12414,363421,7413.0,57.60,...,3098.78,0,1,0.0,53,35.000000,10.851500,6.799000,6.311500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,1972503035,MICHAEL JESSUP,CAPE GIRARDEAU,MO,Obstetrics & Gynecology,393961.33,10857,330005,2448.0,21.00,...,77.59,2,42,25.0,190,493.565586,124.913392,89.406376,84.004145,1
114,1982607370,ANDRES MENCIA,OAKLAND PARK,FL,Geriatric Medicine,2156155.86,19462,767642,2102.0,10.15,...,21259.15,15,187,353.0,1694,2190.454791,1124.673030,875.553880,820.175297,1
115,1982609723,NARAYAN MULAMALLA,OLYMPIA FIELDS,IL,Interventional Cardiology,232272.73,4224,251274,0.0,0.00,...,748.95,1,1,0.0,12,230.000000,77.180000,55.467500,50.728333,1
116,1982706941,MAHMOUD RAHIM,DEARBORN,MI,Internal Medicine,118161.71,1542,50956,60.0,2.94,...,469.07,0,7,25.0,25,64.616331,9.740000,7.636897,7.636897,1


There are only 118 fraud provider in our dataset